## Investigation of a rare variant in FANCI gene: 
chr15:89315318

In [1]:
import hail as hl
import sys
import os

In [2]:
hl.init(tmp_dir=os.path.join(os.environ.get('SCRATCH'), 'hail-tmpdir'),
        local_tmpdir=os.path.join(os.environ.get('SCRATCH'), 'hail-local-tmpdir'),
        default_reference='GRCh38') 

2022-12-01 09:57:03.692 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.3
SparkUI available at http://ac0007:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.105-acd89e80c345
LOGGING: writing to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-sportwgs/preprocessing/joint-with-gts/hail-20221201-0956-0.2.105-acd89e80c345.log


In [3]:
from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
hl.plot.output_notebook()
import numpy as np
import pandas as pd
from functools import reduce
from itertools import chain
from bokeh.plotting import output_notebook, show, figure
from bokeh.palettes import viridis

output_notebook() 

Loading BokehJS ...

Loading BokehJS ...

In [13]:
mt = hl.read_matrix_table(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-sportwgs/data/joint/dense-all.mt'
)

In [17]:
mt = mt.filter_rows(mt.locus.contig == 'chr15')
mt.write(os.path.join(os.environ.get('SCRATCH'), 'chr15_1.mt'))

2022-11-30 20:40:08.612 Hail: INFO: wrote matrix table with 2106682 rows and 789 columns in 445 partitions to /net/ascratch/people/plggosborcz/chr15_1.mt


In [18]:
mt = hl.read_matrix_table(os.path.join(os.environ.get('SCRATCH'), 'chr15_1.mt'))

In [21]:
genes = hl.read_table('/net/pr2/projects/plgrid/plggneuromol/resources/genecode_v32.ht')
genes = genes.filter(hl.is_valid_contig(genes['hg38.knownGene.chrom'], reference_genome='GRCh38'))
hpo = hl.import_table('/net/pr2/projects/plgrid/plggneuromol/resources/hpo.tsv', impute = True, no_header=True)

start = genes['hg38.knownGene.txStart']
stop =  genes['hg38.knownGene.txEnd']
genes = genes.transmute(interval = 
                        hl.locus_interval(genes['hg38.knownGene.chrom'], 
                                          start,
                                          stop,
                                          reference_genome='GRCh38', includes_start=False))

genes = genes.key_by(genes['hg38.kgXref.geneSymbol'])

hpo = hpo.key_by(hpo.f0)
genes = genes.annotate(hpo = hpo.index(genes['hg38.kgXref.geneSymbol'], all_matches = True)['f1'])
genes = genes.key_by(genes.interval)

2022-11-30 20:43:24.237 Hail: INFO: Reading table to impute column types
2022-11-30 20:43:25.108 Hail: INFO: Finished type imputation
  Loading field 'f0' as type str (imputed)
  Loading field 'f1' as type str (imputed)


In [22]:
mt = mt.annotate_rows(
    within_gene = hl.array(
        hl.set(
            genes.index(
                mt.locus, all_matches=True
            )['hg38.kgXref.geneSymbol']
        )
    )
)

In [22]:
mt = mt.filter_rows(
    mt.within_gene.contains('FANCI')
)

In [23]:
mt.write(os.path.join(os.environ.get('SCRATCH'), 'fanci.mt'))

2022-11-30 20:45:29.493 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-30 20:45:30.804 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-30 20:45:32.437 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-30 20:46:35.815 Hail: INFO: wrote matrix table with 2106682 rows and 789 columns in 445 partitions to /net/ascratch/people/plggosborcz/fanci.mt
    Total size: 5.56 GiB
    * Rows/entries: 5.56 GiB
    * Columns: 3.22 KiB
    * Globals: 11.00 B
    * Smallest partition: 24 rows (94.91 KiB)
    * Largest partition:  21961 rows (76.95 MiB)


In [4]:
mt = hl.read_matrix_table(os.path.join(os.environ.get('SCRATCH'), 'fanci.mt'))

In [5]:
mt.count()

(2106682, 789)

In [6]:
mt = mt.filter_rows(hl.agg.any(mt.GT.is_non_ref()))

In [7]:
mt.write(os.path.join(os.environ.get('SCRATCH'), 'fanci-5.mt'))

2022-11-30 21:14:13.764 Hail: INFO: wrote matrix table with 2106682 rows and 789 columns in 445 partitions to /net/ascratch/people/plggosborcz/fanci-5.mt


In [8]:
mt = hl.read_matrix_table(os.path.join(os.environ.get('SCRATCH'), 'fanci-5.mt'))

In [9]:
mt.count()

(2106682, 789)

In [10]:
mt = mt.annotate_cols(group = hl.if_else(
     mt.s.contains('B'), 'sport', hl.if_else(
     mt.s.contains('HG'), '1kg', hl.if_else(
           mt.s.contains('NA'), '1kg', 'GTS'))))

rpmk = hl.read_table('/net/pr2/projects/plgrid/plggneuromol/resources/repeatmasker-extended-keyed.ht')
cov = hl.read_table('/net/pr2/projects/plgrid/plggneuromol/resources/gnomad/gnomad.genomes.r3.0.1.coverage.ht')
cov = cov.filter(cov.over_1 > 0.9)

mt = mt.filter_rows(hl.is_defined(rpmk[mt.locus]), keep = False)
mt = mt.filter_rows(hl.is_defined(cov[mt.locus]), keep = True)

In [11]:
mt.write(os.path.join(os.environ.get('SCRATCH'), 'fanci-6.mt'))

2022-11-30 21:14:59.505 Hail: INFO: Ordering unsorted dataset with network shuffle
2022-11-30 21:16:57.203 Hail: INFO: wrote matrix table with 927205 rows and 789 columns in 445 partitions to /net/ascratch/people/plggosborcz/fanci-6.mt


In [25]:
mt = hl.read_matrix_table(os.path.join(os.environ.get('SCRATCH'), 'fanci-6.mt'))

In [26]:
mt.count()

(927205, 789)

In [ ]:
mt = mt.annotate_cols(group = hl.if_else(
     mt.s.contains('B'), 'sport', hl.if_else(
     mt.s.contains('HG'), '1kg', hl.if_else(
           mt.s.contains('NA'), '1kg', 'control'))))

mt = mt.annotate_rows(dp_qc = hl.agg.group_by(mt.group, hl.agg.stats(mt.DP)),
                      gq_qc = hl.agg.group_by(mt.group, hl.agg.stats(mt.GQ)),
                      hwe = hl.agg.group_by(mt.group, hl.agg.hardy_weinberg_test(mt.GT)))

mt = mt.annotate_rows(n_below_dp_3 = hl.agg.group_by(mt.group, hl.agg.count_where(mt.DP < 3)),
                      n_below_gq_30 = hl.agg.group_by(mt.group, hl.agg.count_where(mt.GQ <30)))


mt = mt.filter_rows((mt.dp_qc.get('GTS', hl.struct(mean=6.0, stdev=0.0, min=0.0, max=0.0, n=0, sum=0.0)).mean > 5) &
                    (mt.dp_qc.get('sport', hl.struct(mean=6.0, stdev=0.0, min=0.0, max=0.0, n=0, sum=0.0)).mean > 5) &
                    (mt.gq_qc.get('GTS', hl.struct(mean=60.0, stdev=0.0, min=0.0, max=0.0, n=0, sum=0.0)).mean > 50) &
                    (mt.gq_qc.get('sport', hl.struct(mean=60.0, stdev=0.0, min=0.0, max=0.0, n=0, sum=0.0)).mean > 50) &
                    (mt.hwe.get('GTS', hl.struct(het_freq_hwe=0.0, p_value=0.5)).p_value > 0.05) &
                    (mt.hwe.get('sport', hl.struct(het_freq_hwe=0.0, p_value=0.5)).p_value > 0.05) &
                    (mt.n_below_dp_3.get('sport', 0) < 3) &
                    (mt.n_below_gq_30.get('sport', 0) < 30) &
                    (mt.n_below_dp_3.get('GTS', 0) < 3) &
                    (mt.n_below_gq_30.get('GTS', 0) <30))

In [29]:
mt.write(os.path.join(os.environ.get('SCRATCH'), 'fanci-7.mt'))

2022-11-30 21:23:55.570 Hail: INFO: wrote matrix table with 835035 rows and 789 columns in 445 partitions to /net/ascratch/people/plggosborcz/fanci-7.mt


In [9]:
mt = hl.read_matrix_table(os.path.join(os.environ.get('SCRATCH'), 'fanci-7.mt'))

In [10]:
gnomad = hl.read_table('/net/pr2/projects/plgrid/plggneuromol/resources/gnomad/gnomad.genomes.v3.1.1.sites.ht/')
cadd = hl.read_table('/net/pr2/projects/plgrid/plggneuromol/resources/cadd.ht')
vep = hl.read_table('/net/pr2/projects/plgrid/plggneuromol/resources/vep38/grch38_context_vep_annotated.ht')

sport_pheno = hl.import_table(
    "/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-sportwgs/data/external-data/sportsmen-pheno.csv",
    impute = True, 
    key='sample_id', 
    quote ="\""
)

poles = hl.read_matrix_table(
    '/net/pr2/projects/plgrid/plggneuromol/resources/polish-genomes/polish-genomes.mt')

2022-12-01 10:29:08.909 Hail: INFO: Reading table to impute column types
2022-12-01 10:29:09.516 Hail: INFO: Finished type imputation
  Loading field 'sport' as type str (imputed)
  Loading field 'type' as type str (imputed)
  Loading field 'age' as type int32 (imputed)
  Loading field 'sample_id' as type str (imputed)


In [11]:
mt = mt.distinct_by_row()
mt = mt.key_rows_by(mt.locus, mt.alleles)
mt = hl.split_multi_hts(mt)
mt = mt.annotate_rows(gnomad_v_3_1 = gnomad[mt.row_key])
mt = mt.annotate_rows(cadd = cadd[mt.row_key]['score_phred'])
mt = mt.annotate_rows(vep = vep[mt.row_key])
mt = mt.annotate_cols(sport_phenotypes = sport_pheno[mt.s])
mt = mt.annotate_rows(polish_af = poles.rows()[mt.row_key]['info'])

In [12]:
mt = mt.annotate_rows(het_non_refs = hl.agg.group_by(mt.group, hl.agg.count_where(mt.GT.is_het())))
mt = mt.annotate_rows(hom_refs = hl.agg.group_by(mt.group, hl.agg.count_where(mt.GT.is_hom_ref())))
mt = mt.annotate_rows(hom_non_refs = hl.agg.group_by(mt.group, hl.agg.count_where(mt.GT.is_hom_var())))

mt = mt.annotate_rows(het_non_refs_e_vs_s = hl.agg.group_by(mt.sport_phenotypes.type, hl.agg.count_where(mt.GT.is_het())))
mt = mt.annotate_rows(hom_refs_e_vs_s = hl.agg.group_by(mt.sport_phenotypes.type, hl.agg.count_where(mt.GT.is_hom_ref())))
mt = mt.annotate_rows(hom_non_refs_e_vs_s = hl.agg.group_by(mt.sport_phenotypes.type, hl.agg.count_where(mt.GT.is_hom_var())))

sport_AC_non_ref = hl.int32(mt.het_non_refs.get('sport', 0) + (mt.hom_non_refs.get('sport', 0)*2))
sport_AC_ref = hl.int32(mt.het_non_refs.get('sport', 0) + (mt.hom_refs.get('sport', 0)*2))
control_AC_non_ref = hl.int32(mt.het_non_refs.get('control', 0) + (mt.hom_non_refs.get('control', 0)*2))
control_AC_ref = hl.int32(mt.het_non_refs.get('control', 0) + (mt.hom_refs.get('control', 0)*2))
speed_AC_non_ref = hl.int32(mt.het_non_refs_e_vs_s.get('speed', 0) + (mt.hom_non_refs_e_vs_s.get('speed', 0)*2))
speed_AC_ref = hl.int32(mt.het_non_refs_e_vs_s.get('speed', 0) + (mt.hom_refs_e_vs_s.get('speed', 0)*2))
endurance_AC_non_ref = hl.int32(mt.het_non_refs_e_vs_s.get('endurance', 0) + (mt.hom_non_refs_e_vs_s.get('endurance', 0)*2))
endurance_AC_ref = hl.int32(mt.het_non_refs_e_vs_s.get('endurance', 0) + (mt.hom_refs_e_vs_s.get('endurance', 0)*2))
gnomad_AC_non_ref = mt.gnomad_v_3_1.freq.AC[2]
gnomad_AC_ref = mt.gnomad_v_3_1.freq.AN[2] - mt.gnomad_v_3_1.freq.AC[2]
polish_AC_non_ref = hl.int32(mt.polish_af.AC[0])
polish_AC_ref = hl.int32(mt.polish_af.AN - mt.polish_af.AC[0])

mt = mt.annotate_rows(fisher_sport_vs_control = hl.fisher_exact_test(sport_AC_non_ref, sport_AC_ref, control_AC_non_ref, control_AC_ref),
                              fisher_e_vs_s = hl.fisher_exact_test(speed_AC_non_ref, speed_AC_ref, endurance_AC_non_ref, endurance_AC_ref),
                              fisher_sport_vs_gnomad = hl.fisher_exact_test(sport_AC_non_ref, sport_AC_ref, gnomad_AC_non_ref, gnomad_AC_ref),
                              fisher_sport_vs_polish = hl.fisher_exact_test(sport_AC_non_ref, sport_AC_ref, polish_AC_non_ref, polish_AC_ref))

mt = mt.drop(mt['a_index'], mt['was_split'])

In [ ]:
mt = mt.transmute_rows(DP_stats = mt.dp_qc,
                             GQ_stats = mt.gq_qc, 
                             hwe_controls_p_value = mt.hwe.get('GTS', hl.struct(het_freq_hwe=0.0, p_value=1)).p_value,
                             hwe_sport_p_value = mt.hwe.get('sport', hl.struct(het_freq_hwe=0.0, p_value=1)).p_value,
                             gnomad_v3_nfe_af = mt.gnomad_v_3_1.freq.AF[2],
                             gnomad_v3_nfe_homozygote_count = mt.gnomad_v_3_1.freq.homozygote_count[2],
                             cadd = mt.cadd,
                             within_gene = mt.within_gene,
                             fisher_sport_vs_control  = mt.fisher_sport_vs_control,
                             fisher_e_vs_s = mt.fisher_e_vs_s ,
                             fisher_sport_vs_gnomad  = mt.fisher_sport_vs_gnomad,
                             fisher_sport_vs_polish = mt.fisher_sport_vs_polish,                         
                             het_sport = mt.het_non_refs.get('sport', 0),
                             het_controls = mt.het_non_refs.get('control', 0),
                             hom_ref_sport = mt.hom_refs.get('sport', 0),
                             hom_ref_controls = mt.hom_refs.get('control', 0),
                             hom_var_sport = mt.hom_non_refs.get('sport', 0),
                             how_var_controls = mt.hom_non_refs.get('control',0),
                             het_endurance = mt.het_non_refs_e_vs_s.get('endurance', 0),
                             het_speed = mt.het_non_refs_e_vs_s.get('speed', 0),
                             hom_ref_endurance = mt.hom_refs_e_vs_s.get('endurance', 0),
                             hom_ref_speed = mt.hom_refs_e_vs_s.get('speed', 0),
                             hom_var_endurance = mt.hom_non_refs_e_vs_s.get('endurance', 0),
                             how_var_speed = mt.hom_non_refs_e_vs_s.get('speed',0),
                             most_severe_consequence = mt.vep.vep.most_severe_consequence,
                             transcript_consequences = mt.vep.vep.transcript_consequences,
                             intergenic_consequences = mt.vep.vep.intergenic_consequences,
                             motif_feature_consequences = mt.vep.vep.motif_feature_consequences,
                             regulatory_feature_consequences = mt.vep.vep.regulatory_feature_consequences,
                             polish_af = mt.polish_af)

mt = mt.select_entries(mt.GT)

mt = mt.make_table()

mt.write(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-sportwgs/results/joint-with-gts/fanci.mt'
)

In [13]:
mt = hl.read_table(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-sportwgs/results/joint-with-gts/fanci.mt'
)

In [6]:
mt.count()

916674

In [14]:
fanci = mt.filter(mt.locus.position == hl.int(89315318))
fanci.count()

1

In [18]:
fanci.write(os.path.join(os.environ.get('SCRATCH'), 'fanci-8.mt'))

2022-12-01 10:38:34.128 Hail: INFO: wrote table with 1 row in 1 partition to /net/ascratch/people/plggosborcz/fanci-8.mt


In [21]:
fanci = hl.read_table(os.path.join(os.environ.get('SCRATCH'), 'fanci-8.mt'))

In [22]:
fanci = fanci.export('fanci.csv')

2022-12-01 11:12:23.127 Hail: INFO: merging 2 files totalling 31.1K...
2022-12-01 11:12:23.144 Hail: INFO: while writing:
    fanci.csv
  merge time: 17.347ms


In [31]:
fanci = pd.read_csv('fanci.csv', sep='\t').transpose()

In [33]:
fanci.to_csv('fanci.csv')